In [1]:
!pip install transformers pydantic bitsandbytes-cuda110 bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 53.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 25.2 MB/s eta 0:00:00


In [2]:
import json

with open("/kaggle/input/general-animal-names/general_animal_names.json") as json_data:
    animal_samples = json.load(json_data)["animals"]

print(animal_samples[:10])

['cat', 'lion', 'fish', 'spider', 'fly', 'butterfly', 'horse', 'bull', 'cow', 'dog']


In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, BitsAndBytesConfig
import pandas as pd
from tqdm import tqdm
from pydantic import BaseModel, Field, ValidationError
from typing import List
import spacy
from spacy.tokens import DocBin, Span
import re
import gc 
import random

random.seed(42)

nlp = spacy.load("en_core_web_sm")

model_dir = "/kaggle/input/llama-3.1/transformers/8b-instruct/2/"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False
)


model = AutoModelForCausalLM.from_pretrained(
    model_dir, 
    quantization_config=bnb_config,
    device_map="auto",
    torch_dtype=torch.float16
)
tokenizer = AutoTokenizer.from_pretrained(model_dir)

model.config.use_cache = False
model.config.pretraining_tp = 1

# # Load CSV files
# animal_df = pd.read_csv("/kaggle/input/animal-names/processed_animal_names.csv")
# needed_animal_classes = ["Mammalia", "Aves", "Arachnida", "Insecta", "Pisces", "Amphibia", "Mollusca", "Crustacea", "Cnidaria"]

# animal_samples = list(animal_df[animal_df["class"].isin(needed_animal_classes)]["common_name"])


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [4]:
system_prompt = """
You are an advanced AI trained in natural language processing and synthetic data generation.
Your task is to read the following animal name and generate 10 unique sentences using given animal name.
Make main focus on diversifying sentences - sentence structures and words.

Make sure to extract the exact and use string of the animal name without any changes in it.
For each sentence, highlight the name of the given animal string by setting "||" around it.
You are not allowed to use words that may have a meaning of the animal except given animal name.
Do not provide any explanations.
Only respond with the JSON structured data, structure of JSON should be strictly as in examples.

### Example 1:
Input: 'bald eagle'

Output:
[
    {
        "bald eagle": [
            "The majestic ||bald eagle|| soars high above the tranquil lake, its keen eyes scanning for prey.",
            "With powerful wings, the ||bald eagle|| glides effortlessly through the morning sky.",
            "A symbol of strength and freedom, the ||bald eagle|| commands attention wherever it flies.",
            "The sharp talons of the ||bald eagle|| make it a formidable hunter among the skies.",
            "The call of the ||bald eagle|| echoes through the valleys, a sound both haunting and beautiful.",
            "Under the golden sunset, the silhouette of the ||bald eagle|| is a breathtaking sight.",
            "The ||bald eagle|| is often seen perched on rocky cliffs, surveying the world below.",
            "The ||bald eagle|| is easily identifiable by its white head and tail feathers.",
            "From a distance, it was hard to tell that was the ||bald eagle||.",
            "||eagle|| enthusiasts pay much attention to ||bald eagle||, admiring its regal presence and hunting prowess."
        ]
    }
]


### Example 2:
Input: 'cow'

Output:
[
    {
        "cow": [
            "In the serene meadow, the ||cow|| grazes peacefully under the warm sun.",
            "Once revered in ancient cultures, the ||cow|| holds symbolic meaning even today.",
            "Researchers study the digestion of the ||cow|| to improve agricultural efficiency.",
            "The ||cow||, known for its gentle demeanor, is a beloved farm animal worldwide.",
            "On the rolling hills, the ||cow|| is a symbol of pastoral beauty.",
            "||Cow|| enthusiasts pay much attention for ||cows|| with exceptional milk production.",
            "Farmers appreciate the ||cow|| not only for milk but also for the role of the ||cow|| in sustainable agriculture.",
            "The bell around the ||cow||'s neck jingles as the ||cow|| moves through the pasture.",
            "From ancient myths to modern-day farming, the ||cow|| has always held a special place in human society, symbolizing abundance and nurturing.",
            "The ||cow||'s milk is used to make cheese, yogurt, and butter, highlighting the ||cow||'s importance in the culinary world."
        ]
    }
]


Continue with the task and stop after generating valid output for the given animal by the user by outputting '### Output ends here.'
Don't forget this strict rules.
"""


In [5]:
from transformers import StoppingCriteria, StoppingCriteriaList
from torch import cuda, LongTensor, FloatTensor
import os

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'
failed_animal_names = []
import json
import re

import json
import re

import json
import re

def extract_json_from_response(response):
    """
    Extracts JSON content from the response string.
    It looks for the first occurrence of '[' or '{' after the keyword "Output:" 
    and uses the last occurrence of the corresponding closing bracket before "### Output ends here.".
    If extraction or parsing fails, returns an empty list.
    """
    try:
        # Locate the start of the output segment and the end marker
        output_index = response.find("\nUser:")
        end_index = response.rfind("Output ends here")
        if output_index == -1 or end_index == -1:
            print("Output or end marker not found")
            return []
        # Consider only the text between "Output:" and "### Output ends here."
        segment = response[output_index:end_index]
        # Find the first occurrence of '[' or '{' in the segment
        match = re.search(r'([\[\{])', segment)
        if not match:
            print("No JSON start character found in segment")
            return []
        start_bracket = match.start()
        json_content = segment[start_bracket:].strip()
        # Depending on the first character, find the last matching closing bracket
        if json_content[0] == '[':
            end_bracket = json_content.rfind(']')
        else:
            end_bracket = json_content.rfind('}')
        if end_bracket == -1:
            print("No closing bracket found")
            return []
        json_content = json_content[:end_bracket+1]
        # Debug print the extracted JSON content
        parsed = json.loads(json_content)
        # Wrap dictionary in a list if needed
        if not isinstance(parsed, list):
            parsed = [parsed]
        return parsed
    except json.JSONDecodeError as e:
        print(f"Failed to decode JSON: {e}")
        return []





def create_stopping_criteria(stop_words, tokenizer, device):
    class StoppingCriteriaSub(StoppingCriteria):
        def __init__(self, stops = [], device=device, encounters = 1):
            super().__init__()
            self.stops = stops = [stop.to(device) for stop in stops]

        def __call__(self, input_ids: LongTensor, scores: FloatTensor) -> bool:
            last_token = input_ids[0][-1]
            for stop in self.stops:
                if tokenizer.decode(stop) == tokenizer.decode(last_token):
                    return True
            return False

    stop_word_ids = [tokenizer(stop_word,
                               return_tensors="pt", 
                               add_special_tokens=False)["input_ids"].squeeze() 
                               for stop_word in stop_words]

    stopping_criteria = StoppingCriteriaList([StoppingCriteriaSub(stops=stop_word_ids)])
    return stopping_criteria


stop_words_list = ["Output ends"]
stopping_criteria = None
if stop_words_list is not None:
    stopping_criteria = create_stopping_criteria(stop_words_list, tokenizer, device)

def write_batch_to_json(entities, start_idx):
    filename = os.path.join("/kaggle/working/generated/", "entity_" + str(start_idx) + ".json")
    with open(filename, "w", encoding='utf-8') as f:
        json.dump(entities, f)
        

def predict_entities_in_batches(test_dataset, model, tokenizer, system_prompt):
    text_generation_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer)

    with torch.no_grad():
        for i in tqdm(range(len(test_dataset)), desc="Processing batches"):

            prompt = test_dataset[i]
            prompt = f"{prompt}"
            chat_input = [
                f"{system_prompt}\nUser: {prompt}"
            ]
            results = text_generation_pipeline(chat_input,
                                               max_new_tokens=400,
                                               do_sample=True,
                                               temperature=1.1,
                                               top_p=0.9)
            gc.collect()
            torch.cuda.empty_cache()
            entities = []
            for result in results:
                generated_text = result[0]['generated_text']
                entity = extract_json_from_response(generated_text)
                if entity == []:
                    failed_animal_names.append(prompt)
                else:
                    entities.append(entity)
            
            write_batch_to_json(entities, i) 


os.makedirs("/kaggle/working/generated/", exist_ok=True)

processed_data = predict_entities_in_batches(animal_samples, model, tokenizer, system_prompt)

Processing batches:   4%|▍         | 10/239 [03:19<1:15:37, 19.81s/it]You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Processing batches:  26%|██▋       | 63/239 [21:39<57:23, 19.57s/it]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Failed to decode JSON: Extra data: line 5 column 1 (char 31)


Processing batches:  28%|██▊       | 66/239 [22:40<58:55, 20.44s/it]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Failed to decode JSON: Expecting ',' delimiter: line 14 column 10 (char 1163)


Processing batches:  33%|███▎      | 78/239 [26:51<54:37, 20.36s/it]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


...

Failed to decode JSON: Extra data: line 5 column 1 (char 28)


Processing batches:  85%|████████▍ | 202/239 [1:10:55<12:59, 21.07s/it]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Failed to decode JSON: Extra data: line 4 column 1 (char 29)


Processing batches: 100%|██████████| 239/239 [1:23:51<00:00, 21.05s/it]

Failed to decode JSON: Expecting ',' delimiter: line 9 column 95 (char 672)


In [6]:
filename = "/kaggle/working/failed_animal_names.json"
with open(filename, "w", encoding='utf-8') as f:
    json.dump({"failed_animal_names": failed_animal_names}, f)


In [7]:
import shutil
shutil.make_archive('/kaggle/working/generated', "zip", '/kaggle/working/generated/')

'/kaggle/working/generated.zip'